# Carga de datos

In [1]:
import pymongo
import folium

In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
#carga de datos:
cliente = pymongo.MongoClient()

In [4]:
cliente.list_database_names()

['admin', 'companies_database', 'config', 'local', 'new_database']

In [5]:
db = cliente.companies_database
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies_database')

In [6]:
colec = db.companies
colec

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies_database'), 'companies')

Exploración del archivo

In [7]:
list(colec.find())[0].keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

La información que me interesa traer es:
- name
- category_code
- number_of_employees
- founded_year
- description
- total_money_raised
- offices

Extracción de datos:

In [8]:
oficinas = colec.find({'offices':{'$not':{'$size':0}}},
                         {'name':True,'_id':0,'category_code':1,'founded_year':1,'description':1 ,'number_of_employees':1, 'offices':1, 'total_money_raised':1})

In [9]:
si_oficinas = list(oficinas)
si_oficinas[:10]

[{'name': 'Wetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'description': 'Technology Platform Company',
  'total_money_raised': '$39.8M',
  'offices': [{'description': '',
    'address1': '710 - 2nd Avenue',
    'address2': 'Suite 1100',
    'zip_code': '98104',
    'city': 'Seattle',
    'state_code': 'WA',
    'country_code': 'USA',
    'latitude': 47.603122,
    'longitude': -122.333253},
   {'description': '',
    'address1': '270 Lafayette Street',
    'address2': 'Suite 505',
    'zip_code': '10012',
    'city': 'New York',
    'state_code': 'NY',
    'country_code': 'USA',
    'latitude': 40.7237306,
    'longitude': -73.9964312}]},
 {'name': 'AdventNet',
  'category_code': 'enterprise',
  'number_of_employees': 600,
  'founded_year': 1996,
  'description': 'Server Management Software',
  'total_money_raised': '$0',
  'offices': [{'description': 'Headquarters',
    'address1': '4900 Hopyard Rd.',
    'address2': 'Suite 310',
    'zip_

In [10]:
df = pd.DataFrame(si_oficinas)
df

,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices
0,Wetpaint,web,47.0,2005.0,Technology Platform Company,$39.8M,"[{'description': '', 'address1': '710 - 2nd Av..."
1,AdventNet,enterprise,600.0,1996.0,Server Management Software,$0,"[{'description': 'Headquarters', 'address1': '..."
2,Zoho,software,1600.0,2005.0,Online Business Apps Suite,$0,"[{'description': 'Headquarters', 'address1': '..."
3,Digg,news,60.0,2004.0,user driven social content website,$45M,"[{'description': None, 'address1': '135 Missis..."
4,Facebook,social,5299.0,2004.0,Social network,$2.43B,"[{'description': 'Headquarters', 'address1': '..."
...,...,...,...,...,...,...,...
13739,Oriact,software,NaN,2009.0,Solutions for software development,$0,"[{'description': '', 'address1': 'LÃ¤rchenweg ..."
13740,Adhunk,advertising,3.0,2009.0,Online Admanager,$0,"[{'description': 'Indian Office', 'address1': ..."
13741,AfterLogic,software,NaN,2002.0,.NET Email Components and Webmail,$0,"[{'description': 'Livingston', 'address1': 'P...."
13742,EnteGreat Solutions,software,NaN,NaN,,$0,"[{'description': '', 'address1': '', 'address2..."


Creación de listas a partir del df

In [11]:
list_address = [json_normalize(i) for i in df['offices']]

<ipython-input-11-8cb739a93598>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  list_address = [json_normalize(i) for i in df['offices']]


In [12]:
list_names = list(df['name'])

In [13]:
#verificamos longitud del json_normalize:

print(len(list_names),len(list_address))

13744 13744


In [14]:
#unimos las dos listas
zip_name_df = list(zip(list_names,list_address))

In [15]:
for i in zip_name_df:
    i[1]['name'] = i[0] #cambiarle a company (después)

In [ ]:
concat_df = pd.DataFrame()
for i in range(len(zip_name_df)):
    concat_df = pd.concat([concat_df, zip_name_df[i][1]])

In [ ]:
#Unes usando 'name' como llave 
# el df con las direcciones expandidas con el df original
df_expandido = pd.merge(df, concat_df, on = 'name')

In [ ]:
df_expandido.head()

# Limpieza de datos

In [ ]:
df = df_expandido
df.dtypes

Se eliminara la columna de offices, pues ya no se necesita.
También se cambiará el orden de varias columnas.

# Exploración

In [ ]:
df['category_code'].unique()

voy a buscar información de empresas en las categorías de:
 - analytics
 - finance
 - ecommerce
 - real_state
 - biotech
 - cleantech
 - health

In [ ]:
df['category_code'].value_counts().plot.bar();

definir DataFrame acotado a los sectores de interes:

In [ ]:
categories = ['software','finance','ecommerce','biotech','cleantech','analytics','real_state']

In [ ]:
df_interest = df[df['category_code']=='analytics']

In [ ]:
df_interest

# Gráficas

### Plot 1

In [ ]:
df_interest['country_code'].value_counts().plot.bar();

### Plot 2

In [ ]:
df_coordinates = df_interest[['latitude','longitude','name']] # cambiarle a company después

In [ ]:
df_coordinates.info()

eliminar aquellos que no tienen valores nulos

In [ ]:
df_coord_clean = df_coordinates.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [ ]:
df_coord_clean

In [ ]:
lat = list(df_coord_clean['latitude'])
lng = list(df_coord_clean['longitude'])


In [ ]:
mapa = folium.Map(tiles='Stamen Terrain')

In [ ]:
for i in range(len(lat)):
    folium.Marker([lat[i], lng[i]], icon=folium.Icon(color='red'),popup = '<b>Estos son popups!!</b>', tooltip = 'tooltip').add_to(mapa)
    
mapa

### Plot 3

In [ ]:
map2 = folium.Map(location=[38, -97], zoom_start=4) # centro geográfico

In [ ]:
for i in range(len(lat)):
    folium.Marker([lat[i], lng[i]], icon=folium.Icon(color='red'),popup = '<b>Estos son popups!!</b>', tooltip = 'tooltip').add_to(map2)
    
map2

### Plot 4

In [ ]:
df_estado =  df_interest[['name','city','state_code','country_code']]
df_usa = df_estado.loc[df_estado['country_code'] == 'USA'] 

In [ ]:
df_usa['state_code'].value_counts().plot.pie(y='number', figsize=(20,10),autopct='%1.1f%%')
#plt.title('Las 15 ciudades con más empresas millenials y menos de 200 empleados');

### Plot 5

In [ ]:
df_usa[df_usa['state_code']=='CA']['city'].value_counts()[:15].plot(kind='barh')
plt.title('Top 15 ciudades con más creación de empresas de ecommerce');

# Notas adicionales:

Existe un 'exodo' de empresas, inversionistas y founders de San Francisco a Austin,TX y Miami,FL

https://techcrunch.com/2021/02/03/miami-wont-be-the-next-silicon-valley-because-we-dont-need-another-one/